Yang akan dilakukan:
1. Hitung petrofisika dengan zone dan parameter yang sudah di pick oleh arum dan gurning (~200 - 300 sumur)
2. Buat MLA model
   a. MLA untuk parameter picking
   b. MLA unutk menentukan nilai petrofiiska
3. Bandingkan hasil MLA 2A, dengan MLA 2B

In [13]:
%config Completer.use_jedi = False

# 1 Petrophysic calculation


In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob

In [15]:
# functions for petrophysical analysis

# Vhsale
def vshale(gr, grmin, grmax):
    return (gr - grmin)/(grmax - grmin)


def phid(rhob, rhoma, rhofl):
    return (rhoma - rhob)/(rhoma - rhofl)

def phin(nphi):
    return nphi

def phit(phid, phin):
    return np.sqrt((phid**2 + phin**2)/2)

def phie(phit, vshale, phish):
    phie_ = phit - (vshale*phish)
    phie_ = np.where(phie_>1, 1, phie_)
    phie_ = np.where(phie_<0, 0, phie_)
    return phie_

def sw(rt, phie, rw, a,m,n):
    sw_ = np.power((a/(phie**m)) * (rw/rt), 1/n)
    sw_ = np.where(sw_>1, 1, sw_)
    sw_ = np.where(sw_<0, 0, sw_)
    return sw_

In [16]:
marker = pd.read_excel('lumpsum.xls')

In [17]:
wells = glob.glob('ARD/*')

# fixing wells name to match the database name

In [18]:
def word_to_vector(word):
    vec = [0]*len('abcdefghijklmnopqrstuvwxyz1234567890')
    for num, i in enumerate('abcdefghijklmnopqrstuvwxyz1234567890'):
        val = word.lower()
        val = val.count(i)
        vec[num] = val 
    return np.array(vec)

In [19]:
well_maker = marker.Well.unique()
wells_data = [well.replace('ARD\\','').replace('.csv', '') for well in wells]

In [20]:
pairs = {}
for w1 in well_maker:
    container = []
    for w2 in wells_data:
        w1v   = word_to_vector(w1)
        w2v   = word_to_vector(w2)
        diff = sum(np.abs(w1v - w2v)) 
        container.append(diff)
    loc = np.argmin(container)
    loc = wells_data[loc]
    pairs[w1] = loc+'.csv'

In [21]:
marker['well_data'] = [pairs[well] for well in marker.Well.values]

## Calculate Petrophysic for Real

In [22]:
wells = marker.well_data.unique()

In [23]:
for well in wells:

    try:

        temp = marker[marker['well_data'] == well]
        well_data = pd.read_csv('ARD/'+well).replace(-999.25, np.nan)

        dept = well_data['DEPT'].values

        dummy_grmin = np.array([np.nan]*len(dept))
        dummy_grmax = np.array([np.nan]*len(dept))
        dummy_phish = np.array([np.nan]*len(dept))
        dummy_rw = np.array([np.nan]*len(dept))

        for zone in temp.values:
            top = zone[6]
            bot = zone[7]
            grmin  = zone[15]
            grmax = zone[14]
            phish  = zone[16]
            rw  = zone[18]

            dummy_grmin = np.where((dept>top)&(dept<bot),grmin, dummy_grmin)
            dummy_grmax = np.where((dept>top)&(dept<bot),grmax, dummy_grmax)
            dummy_phish = np.where((dept>top)&(dept<bot),phish, dummy_phish)
            dummy_rw = np.where((dept>top)&(dept<bot),rw, dummy_rw)

        well_data['grmin'] = dummy_grmin
        well_data['grmax'] = dummy_grmax
        well_data['phish'] = dummy_phish
        well_data['rw'] = dummy_rw

        well_data['vshale'] = vshale(well_data['GR'], well_data['grmin'], well_data['grmax'])
        well_data['phid'] = phid(well_data['RHOB'], 2.65, 1.1)
        well_data['phin'] = phin(well_data['NPHI'])
        well_data['phit'] = phit(well_data['phid'], well_data['phin'])
        well_data['phie'] = phie(well_data['phit'], well_data['vshale'], well_data['phish'])
        well_data['sw'] = sw(well_data['RT'], well_data['phie'], well_data['rw'], 1,2,2)

        well_data.to_csv('calculated/'+well, index=False)
        print('we are successfully calculating for well: ', well)
    except Exception:
        print('!!!!!!!!!! failed to calculate on well:', well)
        print(well_data.columns)

we are successfully calculating for well:  McRae 1.csv
we are successfully calculating for well:  A. Goering 1.csv
we are successfully calculating for well:  Alan C-1.csv
we are successfully calculating for well:  Amstutz 1-8.csv
we are successfully calculating for well:  Ankerholz 'A' 1.csv
!!!!!!!!!! failed to calculate on well: PAYNE 5.csv
Index(['GR', 'NPHI', 'RHOB', 'RT'], dtype='object')
we are successfully calculating for well:  B.B. Ainsworth 13.csv
we are successfully calculating for well:  Beagley 1.csv
we are successfully calculating for well:  Behnke 1-15.csv
we are successfully calculating for well:  Berry 6.csv
we are successfully calculating for well:  Berry 5.csv
we are successfully calculating for well:  Blackhall 5.csv
we are successfully calculating for well:  BNE 19-22.csv
we are successfully calculating for well:  BONNEWELL-CARTER UNIT 18-1.csv
we are successfully calculating for well:  Boye 1-16.csv
we are successfully calculating for well:  BOYE 1-9.csv
!!!!!!!!!